In [4]:
# Pyspark Join Intro:Joining DataFrames is a common and powerful operation in PySpark, 
# allowing you to combine datasets for richer analysis.

# Types of Joins: Inner,Left outer, Self, Right Outer, full outer, left-semi, Left-anti, Cross

# Pyspark Join Performance : Performance is a critical consideration when performing joins in PySpark, 
# especially when dealing with large datasets. 
# Here are key strategies and considerations to optimize the performance of joins in PySpark:
# 1. Data Partitioning : a) Shuffling: Avoid shuffling by partitioning the data.
#                        b) Broadcast Join: When one of the dataframe is small, use broadcast(df_small)
# 2. Use Appropriate Join Types : Avoid cross joins, use left-semi/left-anti whenever fits in.
# 3. Pushdown Filters : Apply filters before join to reduce data that needs to be transformed.
# 4. Avoid Skewness: Use salting technique
# 5. Repartitioning: any column is used in frequent join conditions, repartition("key") to redistribute the data.
# 6. Cache/Persist: when the joined df is used multiple times, cache or persist the data to avoid recomputing.
#                  df1.join(df2,"key").cache()
# 7. Optimize Join Algorithms: a) Sort-Merge Join: works well when both DataFrames are large and sorted on the join keys.
#                              b) Broadcast Hash Join: used when one of the DataFrames is small enough to be broadcasted
#                                 to all nodes. 
# 8. Tuning Spark Configurations: a) Partition Size: Adjust spark.sql.shuffle.partitions based on your data size 
#                                    and cluster resources. The default value might not be optimal for large datasets.
#                                 b) Executor Memory: Ensure that your executors have enough memory to handle the DataFrames
#                        being joined. Too little memory can lead to excessive garbage collection or out-of-memory errors.

Note: you may need to restart the kernel to use updated packages.


In [26]:
#pip install findspark
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Joins').master('local[*]').getOrCreate()

#creating employee dataframe
emp = [(1,"Smith",-1,"2018","10","M",3000),
 (2,"Rose",1,"2010","20","M",4000),
 (3,"Williams",1,"2010","10","M",1000),
 (4,"Jones",2,"2005","10","F",2000),
 (5,"Brown",2,"2010","40","",-1),
 (6,"Brown",2,"2010","50","",-1)
 ]

empColumns = ["emp_id","name","superior_emp_id","year_joined","emp_dept_id","gender","salary"]
empDF = spark.createDataFrame(data=emp,schema=empColumns)
empDF.printSchema()
empDF.show(truncate=False)

#creating Department dataframe
dept = [("Finance",10),
 ("Marketing",20),
 ("Sales",30),
 ("IT",40)
 ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data = dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate = False)
#Inner Join
print("Inner join result")
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"inner").show(truncate=False)
#Left Outer Join
print("Left Outer join result")
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"left").show(truncate=False)
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"leftouter").show(truncate=False)
#Right Outer Join
print("Right Outer join result")
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"right").show(truncate=False)
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"rightouter").show(truncate=False)
# Full Outer Join
print("Full Outer join result")
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "outer").show(truncate = False)
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "full").show(truncate = False)
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "fullouter").show(truncate = False)
# Left Semi Join
print("Left Semi join result")
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "leftsemi").show(truncate = False)
# Left Anti Join
print("Left Anti join result")
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "leftanti").show(truncate = False)
# Self Join
print("Self join result")
from pyspark.sql.functions import col
empDF.alias("emp1").join(empDF.alias("emp2"),col("emp1.superior_emp_id") == col("emp2.emp_id"), "inner") \
.select(col("emp1.emp_id"), col("emp1.name"),col("emp2.emp_id").alias("superior_emp_id"),col("emp2.name").alias("superior_emp_name")) \
.show(truncate = False)
# Using spark.sql
print("Spark SQL result")
empDF.createOrReplaceTempView("EMP")
deptDF.createOrReplaceTempView("DEPT")
joinDF = spark.sql("SELECT * FROM EMP e, DEPT d WHERE e.emp_dept_id == d.dept_id").show(truncate = False) 
joinDF2 = spark.sql("SELECT * FROM EMP e INNER JOIN DEPT d ON e.emp_dept_id == d.dept_id").show(truncate = False)



root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+

+------+-----+---------------+-----------+-----------+------+------+
|emp_id|name |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|6     |Brown|2              |2010       |50         |      |-1    |
+------+-----+---------------+-----------+-----------+------+------+

+----

In [52]:
# Broadcast join
from pyspark.sql.functions import broadcast
empDF.join(broadcast(deptDF),empDF.emp_dept_id == deptDF.dept_id,"inner").show(truncate = False)

# cache/persist:MEMORY_ONLY,MEMORY_AND_DISK, MEMORY_ONLY_SER, MEMORY_AND_DISK_SER, DISK_ONLY, MEMORY_ONLY_2,MEMORY_AND_DISK_2

from pyspark import StorageLevel
cachDf=empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id,"inner").cache()

empDF.persist(StorageLevel.MEMORY_AND_DISK)
empDF.show()

empDF.unpersist()

# repartition and coalesce

print(empDF.rdd.getNumPartitions())  #8
coDF=empDF.coalesce(2)

print(coDF.rdd.getNumPartitions()) #2

print(deptDF.rdd.getNumPartitions()) #8
repDF=deptDF.repartition(10)
print(repDF.rdd.getNumPartitions()) #10


+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+

In [41]:
from pyspark.sql.functions import expr, col

# Assume df1 is skewed on the "key" column
# Adding a salt column to df1 and df2

# Step 1: Create a random salt column in df1
empSalt = empDF.withColumn("salt", (expr("floor(rand() * 5)")))

# Step 2: Create the same salt values in df2
# Here, we need to replicate the rows in df2 across the possible salt values
deptSalt = deptDF.crossJoin(spark.range(0, 5).toDF("salt"))

# Step 3: Perform the join on both the key and the salt columns
result = empSalt.join(deptSalt, (empSalt.emp_dept_id == deptSalt.dept_id) & (empSalt.salt == deptSalt.salt))

# Step 4: Drop the salt column after the join
result.drop("salt").show()


+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

